<a href="https://colab.research.google.com/github/minjeeeko/Popup_store_analysis_recommend/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8C%9D%EC%97%85%EC%8A%A4%ED%86%A0%EC%96%B4_%EC%9E%85%EC%A7%80_%EB%B6%84%EC%84%9D_%EB%B0%8F_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

! pip install pycaret
! pip install scikeras
! pip install eli5

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import os
import datetime as dt
from functools import reduce
import numpy as np
import requests
import json
from tqdm import tqdm
from pycaret.clustering import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
plt.rc('font', family='NanumBarunGothic')
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold
from scikeras.wrappers import KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance
from eli5.formatters.as_dataframe import explain_weights_df
from sklearn.model_selection import RandomizedSearchCV

## 상권분석서비스 (점포)

In [ ]:
path = '/content/drive/MyDrive/홍철있는 홍철팀/Data/sanggwun/'
flist = os.listdir(path)
print(flist)

In [ ]:
jumpo = pd.read_csv(path + '/' + flist[1], index_col = 'Unnamed: 0')
jumpo.head(5)

In [ ]:
dong_code = jumpo[['행정동_코드', '행정동_코드_명']].drop_duplicates().reset_index(drop = True)

dong_code.loc[dong_code['행정동_코드_명']=='면목3?8동','행정동_코드_명'] = '면목3·8동'
dong_code.loc[dong_code['행정동_코드_명']=='상계3?4동','행정동_코드_명'] = '상계3·4동'
dong_code.loc[dong_code['행정동_코드_명']=='종로5?6가동','행정동_코드_명'] = '종로5·6가동'
dong_code.loc[dong_code['행정동_코드_명']=='상계6?7동','행정동_코드_명'] = '상계6·7동'
dong_code.loc[dong_code['행정동_코드_명']=='종로1?2?3?4가동','행정동_코드_명'] = '종로1·2·3·4가동'
dong_code.loc[dong_code['행정동_코드_명']=='금호2?3가동','행정동_코드_명'] = '금호2·3가동'
dong_code.loc[dong_code['행정동_코드_명']=='항동','행정동_코드_명'] = '오류2동'
dong_code.loc[dong_code['행정동_코드_명']=='중계2?3동','행정동_코드_명'] = '중계2·3동'

In [ ]:
jumpo_20221 = jumpo[jumpo['기준_년분기_코드'] == 20221]
jumpo_20222 = jumpo[jumpo['기준_년분기_코드'] == 20222]
jumpo_20223 = jumpo[jumpo['기준_년분기_코드'] == 20223]
jumpo_20224 = jumpo[jumpo['기준_년분기_코드'] == 20224]
jumpo_20231 = jumpo[jumpo['기준_년분기_코드'] == 20231]
jumpo_20232 = jumpo[jumpo['기준_년분기_코드'] == 20232]
jumpo_20233 = jumpo[jumpo['기준_년분기_코드'] == 20233]

In [ ]:
def process_jumpo(jumpo_df):
    cols = ['행정동_코드', '서비스_업종_코드_명', '점포_수', '개업_점포_수', '폐업_점포_수', '프랜차이즈_점포_수']
    jumpo_df = jumpo_df[cols]

    jumpo_df['행정동_코드'] = jumpo_df['행정동_코드'].astype(str)
    jumpo_df['행정동_서비스'] = jumpo_df['행정동_코드'] + '_' + jumpo_df['서비스_업종_코드_명']

    jumpo_df = jumpo_df.drop(['행정동_코드', '서비스_업종_코드_명'], axis = 1)

    return jumpo_df

In [ ]:
quarters = ['20221', '20222', '20223', '20224', '20231', '20232', '20233']
modified_jumpo_dfs = {}

for quarter in quarters:
    jumpo_df = globals()[f'jumpo_{quarter}']
    modified_jumpo_df = process_jumpo(jumpo_df)
    modified_jumpo_dfs[quarter] = modified_jumpo_df

jumpo_20221 = modified_jumpo_dfs['20221']
jumpo_20222 = modified_jumpo_dfs['20222']
jumpo_20223 = modified_jumpo_dfs['20223']
jumpo_20224 = modified_jumpo_dfs['20224']
jumpo_20231 = modified_jumpo_dfs['20231']
jumpo_20232 = modified_jumpo_dfs['20232']
jumpo_20233 = modified_jumpo_dfs['20233']

In [ ]:
jumpo_20221.columns = ['221_점포수', '221_개업점포수', '221_폐업점포수', '221_프랜차이즈점포수', '행정동_서비스']
jumpo_20222.columns = ['222_점포수', '222_개업점포수', '222_폐업점포수', '222_프랜차이즈점포수', '행정동_서비스']
jumpo_20223.columns = ['223_점포수', '223_개업점포수', '223_폐업점포수', '223_프랜차이즈점포수', '행정동_서비스']
jumpo_20224.columns = ['224_점포수', '224_개업점포수', '224_폐업점포수', '224_프랜차이즈점포수', '행정동_서비스']
jumpo_20231.columns = ['231_점포수', '231_개업점포수', '231_폐업점포수', '231_프랜차이즈점포수', '행정동_서비스']
jumpo_20232.columns = ['232_점포수', '232_개업점포수', '232_폐업점포수', '232_프랜차이즈점포수', '행정동_서비스']
jumpo_20233.columns = ['233_점포수', '233_개업점포수', '233_폐업점포수', '233_프랜차이즈점포수', '행정동_서비스']

In [ ]:
lst = [jumpo_20221, jumpo_20222, jumpo_20223, jumpo_20224, jumpo_20231, jumpo_20232, jumpo_20233]
all_jumpo = reduce(lambda left, right: pd.merge(left, right, on='행정동_서비스', how='outer'), lst)
dong_service = all_jumpo.pop('행정동_서비스')
all_jumpo.insert(0, '행정동_서비스', dong_service)
all_jumpo.drop_duplicates(inplace = True)
all_jumpo = all_jumpo.fillna(0)

In [ ]:
# 증감률로 2023-4분기 계산

In [ ]:
all_jumpo['2022_34분기_점포수_증감률'] = ((all_jumpo['224_점포수'] - all_jumpo['223_점포수']) / all_jumpo['223_점포수'] * 100).fillna(0)
all_jumpo['2022_34분기_점포수_증감률'] = all_jumpo['2022_34분기_점포수_증감률'].replace([np.inf, -np.inf], 0)

all_jumpo['2022_34분기_개업점포수_증감률'] = ((all_jumpo['224_개업점포수'] - all_jumpo['223_개업점포수']) / all_jumpo['223_개업점포수'] * 100).fillna(0)
all_jumpo['2022_34분기_개업점포수_증감률'] = all_jumpo['2022_34분기_개업점포수_증감률'].replace([np.inf, -np.inf], 0)

all_jumpo['2022_34분기_폐업점포수_증감률'] = ((all_jumpo['224_폐업점포수'] - all_jumpo['223_폐업점포수']) / all_jumpo['223_폐업점포수'] * 100).fillna(0)
all_jumpo['2022_34분기_폐업점포수_증감률'] = all_jumpo['2022_34분기_폐업점포수_증감률'].replace([np.inf, -np.inf], 0)

all_jumpo['2022_34분기_프랜차이즈점포수_증감률'] = ((all_jumpo['224_프랜차이즈점포수'] - all_jumpo['223_프랜차이즈점포수']) / all_jumpo['223_프랜차이즈점포수'] * 100).fillna(0)
all_jumpo['2022_34분기_프랜차이즈점포수_증감률'] = all_jumpo['2022_34분기_프랜차이즈점포수_증감률'].replace([np.inf, -np.inf], 0)

In [ ]:
jumpo_34_change = all_jumpo[['행정동_서비스', '233_점포수', '2022_34분기_점포수_증감률',
                             '233_개업점포수', '2022_34분기_개업점포수_증감률',
                             '233_폐업점포수', '2022_34분기_폐업점포수_증감률',
                             '233_프랜차이즈점포수', '2022_34분기_프랜차이즈점포수_증감률']]
jumpo_34_change

In [ ]:
jumpo_34_change['234_점포수'] = jumpo_34_change['233_점포수'] + (jumpo_34_change['233_점포수'] *  jumpo_34_change['2022_34분기_점포수_증감률'] / 100)
jumpo_34_change['234_개업점포수'] = jumpo_34_change['233_개업점포수'] + (jumpo_34_change['233_개업점포수'] *  jumpo_34_change['2022_34분기_개업점포수_증감률'] / 100)
jumpo_34_change['234_폐업점포수'] = jumpo_34_change['233_폐업점포수'] + (jumpo_34_change['233_폐업점포수'] *  jumpo_34_change['2022_34분기_폐업점포수_증감률'] / 100)
jumpo_34_change['234_프랜차이즈점포수'] = jumpo_34_change['233_프랜차이즈점포수'] + (jumpo_34_change['233_프랜차이즈점포수'] *  jumpo_34_change['2022_34분기_프랜차이즈점포수_증감률'] / 100)

jumpo234 = jumpo_34_change[['행정동_서비스', '234_점포수', '234_개업점포수', '234_폐업점포수', '234_프랜차이즈점포수']]
jumpo234 = jumpo234.reset_index(drop = True)

In [ ]:
before = all_jumpo[['행정동_서비스', '221_점포수', '221_개업점포수', '221_폐업점포수', '221_프랜차이즈점포수',
                                  '222_점포수', '222_개업점포수', '222_폐업점포수', '222_프랜차이즈점포수',
                                  '223_점포수', '223_개업점포수', '223_폐업점포수', '223_프랜차이즈점포수',
                                  '224_점포수', '224_개업점포수', '224_폐업점포수', '224_프랜차이즈점포수',
                                  '231_점포수', '231_개업점포수', '231_폐업점포수', '231_프랜차이즈점포수',
                                  '232_점포수', '232_개업점포수', '232_폐업점포수', '232_프랜차이즈점포수',
                                  '233_점포수', '233_개업점포수', '233_폐업점포수', '233_프랜차이즈점포수']]

final_jumpo = pd.merge(before, jumpo234, on = '행정동_서비스', how = 'inner')

In [ ]:
final_jumpo['행정동_코드'] = final_jumpo['행정동_서비스'].str.split('_').str[0]
final_jumpo['서비스명'] = final_jumpo['행정동_서비스'].str.split('_').str[1]
final_jumpo = final_jumpo.drop('행정동_서비스', axis = 1)
final_jumpo['행정동_코드'] = final_jumpo['행정동_코드'].astype(int)

In [ ]:
final_jumpo = pd.merge(final_jumpo, dong_code, how = 'left', on = '행정동_코드')
final_jumpo = final_jumpo[['행정동_코드', '행정동_코드_명', '서비스명',
                           '221_점포수', '221_개업점포수', '221_폐업점포수', '221_프랜차이즈점포수',
                           '222_점포수', '222_개업점포수', '222_폐업점포수', '222_프랜차이즈점포수',
                           '223_점포수', '223_개업점포수', '223_폐업점포수', '223_프랜차이즈점포수',
                           '224_점포수', '224_개업점포수', '224_폐업점포수', '224_프랜차이즈점포수',
                           '231_점포수', '231_개업점포수', '231_폐업점포수', '231_프랜차이즈점포수',
                           '232_점포수', '232_개업점포수', '232_폐업점포수', '232_프랜차이즈점포수',
                           '233_점포수', '233_개업점포수', '233_폐업점포수', '233_프랜차이즈점포수',
                           '234_점포수', '234_개업점포수', '234_폐업점포수', '234_프랜차이즈점포수']]

In [ ]:
final_jumpo.to_csv(path + '상권서비스_점포_최종_0111.csv', index = False)

In [ ]:
path = '/content/drive/MyDrive/홍철있는 홍철팀/Data/데이터최종본/'
점포데이터 = '상권서비스_점포_진짜최종(240116).csv'
점포 = pd.read_csv(path + 점포데이터, index_col = 0)

In [ ]:
점포_221 = 점포[['행정동코드', '행정동', '대분류', '221_점포수', '221_개업점포수', '221_폐업점포수','221_프랜차이즈점포수']]
점포_222 = 점포[['행정동코드', '행정동', '대분류', '222_점포수', '222_개업점포수', '222_폐업점포수','222_프랜차이즈점포수']]
점포_223 = 점포[['행정동코드', '행정동', '대분류', '223_점포수', '223_개업점포수', '223_폐업점포수','223_프랜차이즈점포수']]
점포_224 = 점포[['행정동코드', '행정동', '대분류', '224_점포수', '224_개업점포수', '224_폐업점포수','224_프랜차이즈점포수']]
점포_231 = 점포[['행정동코드', '행정동', '대분류', '231_점포수', '231_개업점포수', '231_폐업점포수','231_프랜차이즈점포수']]
점포_232 = 점포[['행정동코드', '행정동', '대분류', '232_점포수', '232_개업점포수', '232_폐업점포수','232_프랜차이즈점포수']]
점포_233 = 점포[['행정동코드', '행정동', '대분류', '233_점포수', '233_개업점포수', '233_폐업점포수','233_프랜차이즈점포수']]
점포_234 = 점포[['행정동코드', '행정동', '대분류', '234_점포수', '234_개업점포수', '234_폐업점포수','234_프랜차이즈점포수']]

In [ ]:
def process_data(df, quarter):
    df['분기'] = [quarter] * len(df)
    df.columns = ['행정동코드', '행정동', '대분류', '점포수', '개업점포수', '폐업점포수', '프랜차이즈점포수', '분기']
    return df

quarters = ['2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4']
dataframes = [점포_221, 점포_222, 점포_223, 점포_224, 점포_231, 점포_232, 점포_233, 점포_234]

for i, quarter in enumerate(quarters):
    dataframes[i] = process_data(dataframes[i], quarter)

점포_221, 점포_222, 점포_223, 점포_224, 점포_231, 점포_232, 점포_233, 점포_234 = dataframes

In [ ]:
점포_221 = 점포_221.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_221['분기'] = ['2022Q1'] * len(점포_221)

점포_222 = 점포_222.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_222['분기'] = ['2022Q2'] * len(점포_222)

점포_223 = 점포_223.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_223['분기'] = ['2022Q3'] * len(점포_223)

점포_224 = 점포_224.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_224['분기'] = ['2022Q4'] * len(점포_224)

In [ ]:
점포_231 = 점포_221.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_231['분기'] = ['2023Q1'] * len(점포_231)

점포_232 = 점포_232.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_232['분기'] = ['2023Q2'] * len(점포_232)

점포_233 = 점포_233.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_233['분기'] = ['2023Q3'] * len(점포_233)

점포_234 = 점포_234.groupby(['행정동코드', '행정동', '대분류']).sum().reset_index()
점포_234['분기'] = ['2023Q4'] * len(점포_234)

In [ ]:
data = pd.concat([점포_221, 점포_222, 점포_223, 점포_224, 점포_231, 점포_232, 점포_233, 점포_234], axis = 0)

In [ ]:
data = pd.read_csv(path + '점포_0116.csv')
data = data.pivot_table(index = ['행정동코드', '행정동', '분기'], columns = ['대분류'], values = ['점포수', '개업점포수', '폐업점포수', '프랜차이즈점포수'], fill_value = 0).reset_index()

In [ ]:
data.columns = ['행정동코드', '행정동', '분기'] + [f"{col[1]}_{col[0]}" if col[0] != '' else col[1] for col in data.columns][3:]
data.to_csv(path + '점포데이터_최종(01161252).csv', index = False)

## 매출데이터

In [ ]:
매출데이터 = '상권분석_매출_최종(230116).csv'
매출 = pd.read_csv(path + 매출데이터)

In [ ]:
cols = ['당월_매출_금액', '주중_매출_금액', '주말_매출_금액',
       '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']
매출 = 매출.pivot_table(index = ['행정동코드', '행정동'], columns = ['분기', '서비스_업종_코드_명'], values = cols, fill_value = 0).reset_index()

In [ ]:
매출.columns = ['행정동코드', '행정동'] + [f"{col[2]}_{col[1]}_{col[0]}" if col[1] != '' else col[2] for col in 매출.columns][2:]
매출.to_csv(path + '매출_전처리_완성.csv', index = False)

## 날씨 데이터 전처리 (평균 기온, 최대 강수량)

In [ ]:
path = r'/content/drive/MyDrive/홍철있는 홍철팀/Data/weather'
flist = os.listdir(path)
print(flist)

In [ ]:
station = pd.read_csv(path + '/' + flist[1], encoding = 'cp949')
station = station[['지점', '시작일', '종료일', '지점명', '위도', '경도']]
station = station.sort_values(by='시작일', ascending=False).drop_duplicates(subset='지점')
station.drop(['시작일', '종료일'], axis = 1, inplace = True)

In [ ]:
def read_and_merge(file_path, station):
    weather_df = pd.read_csv(file_path, encoding='cp949')
    merged_df = pd.merge(weather_df, station, how='left', on='지점')
    return merged_df

merged_weather = pd.DataFrame()

for f in flist[2:]:
    file_path = path + '/' + f
    merged_weather = pd.concat([merged_weather, read_and_merge(file_path, station)], axis=0)

merged_weather.drop('지점명_y', axis = 1, inplace = True)
merged_weather.columns = ['지점', '지점명', '일시', '기온', '강수량', '위도', '경도']
merged_weather

In [ ]:
merged_weather['일시'] = pd.to_datetime(merged_weather['일시'])
merged_weather['달'] = merged_weather['일시'].dt.month

merged_weather['분기'] = merged_weather['달'].apply(lambda x: 1 if x in [1, 2, 3]
                                            else (2 if x in [4, 5, 6]
                                            else (3 if x in [7, 8, 9]
                                            else 4)))
merged_weather.drop(['일시', '달'], axis = 1, inplace = True)

first = merged_weather[merged_weather['분기'] == 1]
second = merged_weather[merged_weather['분기'] == 2]
third = merged_weather[merged_weather['분기'] == 3]
fourth = merged_weather[merged_weather['분기'] == 4]

In [ ]:
Qs = ['first', 'second', 'third', 'fourth']
mean_dfs = []

for Q in Qs:
    df = pd.DataFrame(eval(f'{Q}.groupby(["지점", "분기"])["기온"].mean()'))
    df.columns = ['평균기온']
    mean_dfs.append(df)

first_mean, second_mean, third_mean, fourth_mean = mean_dfs

max_dfs = []

for Q in Qs:
    df = pd.DataFrame(eval(f'{Q}.groupby(["지점", "분기"])["강수량"].max()'))
    df.columns = ['최고강수량']
    max_dfs.append(df)

first_max, second_max, third_max, fourth_max = max_dfs

In [ ]:
first_weather = pd.concat([first_mean, first_max], axis = 1)
first_weather = pd.merge(first_weather, station, how = 'left', on = '지점')

second_weather = pd.concat([second_mean, second_max], axis = 1)
second_weather = pd.merge(second_weather, station, how = 'left', on = '지점')

third_weather = pd.concat([third_mean, third_max], axis = 1)
third_weather = pd.merge(third_weather, station, how = 'left', on = '지점')

fourth_weather = pd.concat([fourth_mean, fourth_max], axis = 1)
fourth_weather = pd.merge(fourth_weather, station, how = 'left', on = '지점')

In [ ]:
first_weather.to_csv(path + '/1분기_날씨.csv')
second_weather.to_csv(path + '/2분기_날씨.csv')
third_weather.to_csv(path + '/3분기_날씨.csv')
fourth_weather.to_csv(path + '/4분기_날씨.csv')

## 팝업스토어 주소 -> 좌표(위경도) 변환

In [ ]:
path = './팝업스토어_리스트(전체) .csv'
popup = pd.read_csv(path)
popup = popup[['이름', '주소']]

apiurl = "http://api.vworld.kr/req/address?"
api_key = "B498C9F9-5196-3554-AEB4-930CECF47CFE" # 본인 인증키로 변경

addresses = list(popup['주소'])

df = pd.DataFrame(columns=['주소', '위도', '경도'])
error_addresses = []

In [ ]:
for address in tqdm(addresses):
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }

    try:
        response = requests.get(apiurl, params=params)
        response.raise_for_status()  # Check if the request was successful

        json_data = response.json()
        경도 = json_data['response']['result']['point']['x']
        위도 = json_data['response']['result']['point']['y']

        df = df.append({'주소': address, '위도': 위도, '경도': 경도}, ignore_index=True)
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh} for address: {address}")
        error_addresses.append({'주소': address, '에러': f"HTTP Error: {errh}"})
    except requests.exceptions.RequestException as err:
        print(f"Request Exception: {err} for address: {address}")
        error_addresses.append({'주소': address, '에러': f"Request Exception: {err}"})
    except Exception as e:
        print(f"An unexpected error occurred for address: {address}, Error: {e}")
        error_addresses.append({'주소': address, '에러': f"Unexpected Error: {e}"})

error_df = pd.DataFrame(error_addresses)

In [ ]:
df.to_csv('/content/drive/MyDrive/홍철있는 홍철팀/Data/popup_위경도.csv', index_col = False)

## 기준 행정동 구하기

In [ ]:
path = r'/content/gdrive/MyDrive/홍철있는 홍철팀/Data/행정동'
flist = os.listdir(path)
print(flist)

In [ ]:
추정매출 = pd.read_csv(path + '/' + flist[0], index_col = 'Unnamed: 0')
공시지가 = pd.read_csv(path + '/' + flist[1])
행정동코드 = pd.read_csv(path + '/' + flist[2])
날씨 = pd.read_csv(path + '/' + flist[3])
팝업스토어 = pd.read_csv(path + '/' + flist[4])
대중교통 = pd.read_csv(path + '/' + flist[5])

In [ ]:
set(행정동코드['0']) - set(추정매출['0'])

In [ ]:
set(행정동코드['0']) - set(날씨['ADM_NM'])

In [ ]:
len(set(행정동코드['0']) - set(팝업스토어['행정동']))

In [ ]:
set(행정동코드['0']) - set(대중교통 ['행정동명'])

## 최종데이터 입지분석(클러스터링 수행)

In [ ]:
d2022 = pd.read_csv('/content/drive/MyDrive/홍철있는 홍철팀/데이터 분석/클러스터링/fin_2022_df.csv')
d2023 = pd.read_csv('/content/drive/MyDrive/홍철있는 홍철팀/데이터 분석/클러스터링/fin_2023_df.csv')

In [ ]:
columns_to_drop = [col for col in d2022.columns if '매출' in col or '지출' in col]
d2022.drop(columns=columns_to_drop, inplace=True)

columns_to_drop = [col for col in d2023.columns if '매출' in col or '지출' in col]
d2023.drop(columns=columns_to_drop, inplace=True)

In [ ]:
d2022['전체'] = (d2022['가전디지털'] + d2022['게임'] + d2022['금융']
               + d2022['리빙'] + d2022['뷰티'] + d2022['연예']
               + d2022['음료'] + d2022['음식'] + d2022['취미여가']
               + d2022['캐릭터'] + d2022['콘텐츠'] + d2022['패션'])

d2022 = d2022.drop(['가전디지털', '게임', '금융', '리빙', '뷰티', '연예', '음료', '음식', '취미여가', '캐릭터', '콘텐츠', '패션'], axis = 1)

d2023['전체'] = (d2023['가전디지털'] + d2023['게임'] + d2023['금융']
               + d2023['리빙'] + d2023['뷰티'] + d2023['연예']
               + d2023['음료'] + d2023['음식'] + d2023['취미여가']
               + d2023['캐릭터'] + d2023['콘텐츠'] + d2023['패션'])

d2023 = d2023.drop(['가전디지털', '게임', '금융', '리빙', '뷰티', '연예', '음료', '음식', '취미여가', '캐릭터', '콘텐츠', '패션'], axis = 1)

In [ ]:
corr_matrix = d2022.corr()
correlation_with_all = corr_matrix['전체']

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('상관관계 히트맵 - "전체"')
plt.show()

In [ ]:
corr_matrix = d2023.corr()
correlation_with_all = corr_matrix['전체']

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('상관관계 히트맵 - "전체"')
plt.show()

In [ ]:
d2022_col = d2022[['행정동코드', '행정동_x', '평균생활인구수', '총직장인구수', '음식점업_점포수', '승객수', '총직장인구수_평균',
                   'MJ_2030_평균', 'MJ_4050_평균', 'WJ_2030_평균', 'WJ_4050_평균', '평균생활인구수_평균',
                   'M_2030_평균', 'M_4050_평균', 'W_2030_평균', 'W_4050_평균', '전체']]

d2023_col = d2023[['행정동코드', '행정동_x', '평균생활인구수', '총직장인구수', '음식점업_점포수', '승객수', '총직장인구수_평균',
                   'MJ_2030_평균', 'MJ_4050_평균', 'WJ_2030_평균', 'WJ_4050_평균', '평균생활인구수_평균',
                   'M_2030_평균', 'M_4050_평균', 'W_2030_평균', 'W_4050_평균', '전체']]

In [ ]:
norm = d2023_col.drop(['행정동코드', '행정동_x'], axis = 1)

scaler = MinMaxScaler()
norm_data = scaler.fit_transform(norm)

norm_2023 = pd.DataFrame(norm_data, columns=norm.columns)
final_2023 = pd.concat([d2023_col[['행정동코드', '행정동_x']], norm_2023], axis = 1)

In [ ]:
norm22 = d2022_col.drop(['행정동코드', '행정동_x'], axis = 1)

scaler = MinMaxScaler()
norm22_data = scaler.fit_transform(norm22)

norm_2022 = pd.DataFrame(norm22_data, columns=norm22.columns)
final_2022 = pd.concat([d2022_col[['행정동코드', '행정동_x']], norm_2022], axis = 1)

In [ ]:
# trainset23 = setup(data = final_2023)
trainset22 = setup(data = final_2022)

In [ ]:
kmeans = create_model('kmeans')

In [ ]:
evaluate_model(kmeans)

In [ ]:
kmean_results = assign_model(kmeans)
kmean_results['Cluster'].value_counts()

In [ ]:
kmean_results.to_csv('/content/drive/MyDrive/홍철있는 홍철팀/데이터 분석/클러스터링/kmeans22.csv', index = False)

In [ ]:
dat22= pd.read_csv('/content/drive/MyDrive/홍철있는 홍철팀/데이터 분석/클러스터링/kmeans22.csv')
dat23=pd.read_csv('/content/drive/MyDrive/홍철있는 홍철팀/데이터 분석/클러스터링/kmeans23.csv')

In [ ]:
cluster_labels = kmeans.labels_

for cluster_num in range(kmeans.n_clusters):
    cluster_data = dat22[cluster_labels == cluster_num]
    print(f"클러스터 {cluster_num}에 속한 데이터:")
    print(cluster_data)

In [ ]:
for cluster_num in range(kmeans.n_clusters):
    cluster_data = dat22[cluster_labels == cluster_num]
    print(f"\n클러스터 {cluster_num} 특성 요약 통계량:")
    print(cluster_data.describe())

In [ ]:
cl22=dat22['Cluster']
cl23=dat23['Cluster']
cluster22=pd.concat([d2022_col,cl22],axis=1)
cluster23=pd.concat([d2023_col,cl23],axis=1)

In [ ]:
group=cluster22.groupby('Cluster').agg('mean','max')

## 입지 추천

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/홍철있는 홍철팀/데이터 분석/추천/fin_2023_co.csv')

In [ ]:
# 뷰티
data = data.drop(['패션','가전디지털','게임', '금융', '리빙', '연예', '음료', '음식','취미여가','캐릭터','콘텐츠'], axis = 1)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y)

feature_importances = rf_model.feature_importances_
top_30_features = X.columns[np.argsort(feature_importances)[::-1]][:30]

In [ ]:
top30 = data[top_30_features]

scaler = StandardScaler()
top30[top30.columns] = scaler.fit_transform(top30)
top_scaled = scaler.fit_transform(top30)
y = data['뷰티']

In [ ]:
def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, dropout_rate=0.0):
  model = Sequential()
  model.add(InputLayer(input_shape=(X_scaled.shape[1],)))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation = 'relu'))
    if dropout_rate > 0.0:
      model.add(Dropout(dropout_rate))
  model.add(Dense(1))
  optimizer = Adam(learning_rate = learning_rate)
  model.compile(loss='mean_squared_error', metrics=['mae'], optimizer = optimizer)
  return model

keras_model = KerasRegressor(model = build_model, dropout_rate=0.0,
                             learning_rate=0.001, n_hidden=0, n_neurons=1)

keras_model.fit(top_scaled, y, epochs = 5,
                validation_split = 0.1,
                batch_size=16,
                callbacks = [EarlyStopping(patience = 10)])

In [ ]:
params = {
    "n_hidden" : [0, 1, 2, 3, 4, 5],
    "n_neurons" : np.arange(1, 100).tolist(),
    "learning_rate" : [0.001, 0.01, 0.1, 0.5],
    "dropout_rate": [0.1, 0.2, 0.3, 0.4, 0.5],
    "batch_size": [16, 32, 64, 128]
}

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

In [ ]:
random_search = RandomizedSearchCV(keras_model, params, n_iter = 10, cv = 4)
random_search.fit(top_scaled, y)

In [ ]:
print(random_search.best_score_)
print(random_search.best_params_)

In [ ]:
def build_model(n_hidden = 4, n_neurons = 40, learning_rate = 0.001, dropout_rate=0.4):
  model = Sequential()
  model.add(InputLayer(input_shape=(top_scaled.shape[1],)))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation = 'relu'))
    if dropout_rate > 0.0:
      model.add(Dropout(dropout_rate))
  model.add(Dense(1))
  optimizer = Adam(learning_rate = learning_rate)
  model.compile(loss='mean_squared_error', metrics=['mae'], optimizer = optimizer)
  return model

keras_model = KerasRegressor(model = build_model, dropout_rate=0.4,
                             learning_rate=0.001, n_hidden=4, n_neurons=40)

keras_model.fit(top_scaled, y, epochs =100,
                validation_split = 0.1,
                batch_size=64,
                callbacks = [EarlyStopping(patience = 10)])

In [ ]:
perm = PermutationImportance(keras_model, random_state=1).fit(top_scaled,y)

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_rows',None)
weights_df = explain_weights_df(perm, feature_names=top30.columns.tolist())

In [ ]:
new_df = pd.DataFrame()

for index, row in weights_df.iterrows():
  feature = row['feature']
  weight = row['weight']
  new_df[feature] = top30[feature] * weight

df = data[['행정동코드', '행정동', '뷰티']]
new_df2 = new_df.sum(axis = 1)
result = pd.concat([df, new_df2], axis = 1)
result.sort_values(by = 0, ascending = False)

In [ ]:
weight_dong_array = np.dot(top_scaled,(np.array(weights_df['weight']))).reshape(424, 1)
dong_lst = [item for sublist in weight_dong_array for item in sublist]
dong_weight = pd.DataFrame(dong_lst)

final = pd.concat([data[['행정동코드', '행정동']], dong_weight], axis = 1)
final.sort_values(by = 0, ascending = False)